# Requirments
- Python 3.11.9
- install the latest backtesting.py version `pip install git+https://github.com/kernc/backtesting.py.git`
- install additional deps `pip install yfinance pandas ipykernal`
    - you only need ipykernel if you are running a jupyter notebook
- TA Lib
    - `brew install ta-lib` - need ta-lib c library to work with ta-lib python
    - `pip install numpy==1.26.4` - need to ensure that we are using a numpy version lower then 2.0

## Strategy 
Buy when the 20 EMA Crosses over the 50 EMA
Sell when the 20 EMA Crosses Under the 50 EMA

In [1]:
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd
# import numpy as np
# print(np.__version__)
import talib

# import numpy as np
# print(np.__version__)

%load_ext autoreload
%autoreload 2

In [ ]:
class EMACrossStrategy(Strategy):
    n1 = 20  # fast EMA period
    n2 = 50  # slow EMA period

    def init(self):
        # Calculate EMAs
        self.ema1 = self.I(
            lambda x: pd.Series(x).ewm(span=self.n1, adjust=False).mean(),
            self.data.Close,
        )
        self.ema2 = self.I(
            lambda x: pd.Series(x).ewm(span=self.n2, adjust=False).mean(),
            self.data.Close,
        )

    def next(self):
        if crossover(self.ema1, self.ema2):
            self.buy()
        elif crossover(self.ema2, self.ema1):
            self.sell()

In [ ]:
# Download the Data
start_date = "2023-08-12"
end_date = "2024-08-11"
data = yf.download("BTC-USD", start=start_date, end=end_date)
data

In [ ]:
# Create and run the backtest
bt = Backtest(data, EMACrossStrategy, cash=100000, commission=0.002)
stats = bt.run()

# Print results
print(stats)
# Plot the result
bt.plot()